In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid")

# Evaluation Playground

Start building the evaluation for the reception study here.
Move more mature stuff to specific notebooks and scripts later.

## Simulation and Result Pre-Processing

Doing the stuff outside python:

In [ ]:
# run simulation
!env  -C scenario PATH="/scratch/buse/sumo-1_6_0/bin:$PATH" ../lib/veins/bin/veins_run -u Cmdenv -c Default
# convert to csv
!lib/veins_scripts/eval/opp_vec2longcsv.sh scenario/results/Default-\#0.vec > scenario/results/Default-\#0.vec.csv
!lib/veins_scripts/eval/opp_sca2longcsv.sh scenario/results/Default-\#0.sca > scenario/results/Default-\#0.sca.csv
!ls -hl scenario/results/Default-\#0.*.csv

## Data Reading

In [ ]:
vec = (
        pd.read_csv(
        "scenario/results/Default-#0.vec.csv",
        sep=" ",
        names=["vecid", "module", "signal", "event", "time", "value"],
    )
    .pipe(lambda df: df.merge(df.module.str.extract(r"[^.]+\.node\[(?P<hostnr>\d+)\]\.(?P<submodule>.*)"), left_index=True, right_index=True))
    .drop(columns=['module'])
    .assign(signal=lambda df: df.signal.str.replace(":vector", ""))
    .astype({"submodule": "category", "signal": "category", "hostnr": int})
)
vec.info()
vec.head()

In [ ]:
vec.groupby(["hostnr", "signal"]).event.count().unstack()

## First Insights

Explore the RSS and SNR over time and (later) distance.

I have added recording points into `veins::Decider80211p::processSignalEnd` method.
There I record some signal properties, regardless of wheter the signal could be decoded or even detected.
That would not have been possible in the MAC layer, as that only knows about successfully decoded frames.
However, I'll only get valid SNR values for signals that were at least detected -- for others the Decider stops early and does not even compute it.

In [ ]:
receptions = (
    vec.query("hostnr == 0 and signal in ('RSSIdBm', 'SNR', 'Correct', 'Detected')")
    [['time', 'signal', 'value']]
    .pivot(index=["time"], columns=["signal"], values="value")
    .assign(SNRdB=lambda df: 10 * np.log10(df.SNR))
    .astype({"Correct": bool, "Detected": bool})
    .reset_index()
    [['time', 'Detected', 'Correct', 'RSSIdBm', 'SNRdB']]
)
receptions.head()

### Detection Threshold

Message detection stops at around 1110 s, so around 1120 meters of distance (vehicles start with 1 m between them and diverge with 1 m/s).

In [ ]:
sns.violinplot(data=receptions, y="time", x="Detected")
receptions.groupby("Detected").time.describe()

There actually is a hard cut-off with no stochastic process in beween.

**Note**: There is no influence of noise or interference here, this boundary is purely based on the RSS of the incoming signal itself and the receiver config.
However, a previous message that the receiver is trained on will affect detection (only one signal receptable at the time, no frame capturing).

In [ ]:
detection = receptions.assign(second=lambda df: df.time.astype(int)).groupby("second").Detected.sum().reset_index()
detection_change_boundaries = detection.query("Detected < 100 and Detected > 0").second
fig, ax = plt.subplots()
sns.scatterplot(data=detection, x="second", y="Detected", ax=ax)
ax.set_xlim(left=detection.query("Detected == 100").second.iloc[-1] - 20, right=detection.query("Detected == 0").second.iloc[0] + 20)

### SNR and RSS

Signals will only be detected if they are above the `minPowerLevel` setting (of -98 dBm, indicated by the dotted horizontal line).
Signals below that will still be processed by the Decider, but not even considered for decoding.
Thus, there are no values for the SNR for that.

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
sns.lineplot(
    data=receptions.melt(id_vars=["time", "Detected", "Correct"], var_name="signal", value_name="value"),
    x="time",
    y="value",
    hue="signal",
    style="Correct",
    ax=ax,
)
ax.hlines(y=-98, xmin=receptions.time.min(), xmax=receptions.time.max(), color="grey", linestyle="dotted")
ax.vlines(x=receptions.time.iloc[receptions.SNRdB.idxmin()], ymin=receptions.RSSIdBm.min(), ymax=receptions.SNRdB.max(), color="grey", linestyle="dotted")
ax.set_ylabel("RSS [dBm] / SNR [dB]")

### Decodability

Signals start to become not decodable (aka not `correct`) at around 420 s.
Though it is only spurious at the time -- most messages still come through.
Only at arount 580 s there are no more decodable messages.

When looking at the relation beween decodability and RSS/SNR, a similar pattern is visible.
Just note that the x axis appears flipped now, as distance increases over time while RSS (and thus also SNR) decreases.

These patterns appear to match the plots in bloessl2019case and the original NIST error model paper (for the configured QPSK 1/2 and 500 Byte frames, as we do here).

In [ ]:
pdr = receptions.assign(second=lambda df: df.time.astype(int)).groupby("second").agg({"Correct": "sum", "RSSIdBm": "mean", "SNRdB": "mean"}).reset_index()
pdr_change_boundaries = pdr.query("Correct < 100 and Correct > 0")

fig, (left, mid, right) = plt.subplots(1, 3, figsize=(18, 5), sharey=True, constrained_layout=True)
sns.scatterplot(data=pdr, x="second", y="Correct", ax=left)
sns.scatterplot(data=pdr, y="Correct", x="RSSIdBm", ax=mid)
sns.scatterplot(data=pdr, y="Correct", x="SNRdB", ax=right)
left.set_xlim(left=pdr_change_boundaries.second.min() - 10, right=pdr_change_boundaries.second.max() + 10)
mid.set_xlim(left=pdr_change_boundaries.RSSIdBm.min() - 0.5, right=pdr_change_boundaries.RSSIdBm.max() + 0.5)
right.set_xlim(left=pdr_change_boundaries.SNRdB.min() - 0.5, right=pdr_change_boundaries.SNRdB.max() + 0.5)

## TODO: Influence of Interference

Now that I know about the basic behavior, I want to find out how increased interference could change the results.
Note that Veins treats interference and noise mostly the same (except for some reporting), so I could also just adapt the noise to get an impression of what would change.

Main Questions:

- How much interference/noise is needed to significantly shift the reception behavior?
- How much interference can there be (assuming CSMA/CA works)?
- And finally: at what distance will a signal be so weak that it can not interfere with the reception of another signal anymore?